In [ ]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

## Absolute MMR Assignment Problem

Basado en Jordi Pereira, Igor Averbakh, Exact and heuristic algorithms for the interval data robust assignment problem, In Computers & Operations Research, Volume 38, Issue 8, 2011, Pages 1153-1163, ISSN 0305-0548, https://doi.org/10.1016/j.cor.2010.11.009.

Utilizaremos el siguiente modelo

$\min \sum_{(u,v)\in E} c_{uv}^+ x_{uv} - \theta$


s.t.

$\theta \leq \sum_{(u,v)\in E} c_{uv}^- y_{uv} + \sum_{(u,v)\in E} y_{uv} (c_{uv}^+ - c_{uv}^-) x_{uv}\qquad \forall Y\in A$

$\sum_{u\in U} x_{uv} = 1, \qquad v\in V$;

$\sum_{v\in V} x_{uv} = 1, \qquad u\in U$;

$x_{uv}\in \{0,1\}, \qquad u\in U, v\in V$.

Nótese que la restricción que calcula $\theta$ hace uso de diversas propiedades del problema (sabemos que el escenario que maximiza la diferencia entre dos soluciones es un escenario extremo de características concretas y que una de esas soluciones define la peor alternativa y por tanto el worst-case regret de una solución $X$.

En realidad es más conveniente implementar como:

$\min \theta$


s.t.

$\theta \geq \sum_{(u,v)\in E} c_{uv}^s x_{uv} - c_{uv}^s x_{uv}^{\ast}\forall s\in S$

y específicamente definir $s$ para una solución $x$ como su caso extremo. Esto es, $c_{uv}^s= c_{uv}^+$ si $x_{uv}=1$ y $c_{uv}^s= c_{uv}^-$ en el resto de los casos.

In [ ]:
function doInstanciaAleatoria(n::Int64)
    cplus = rand(1:1000000, n*n)
    cminus=map(x -> rand(1:x), cplus)
    return reshape(cminus,n,n),reshape(cplus,n,n)
end

In [ ]:
srand(0)
cminus,cplus=doInstanciaAleatoria(10)

In [ ]:
function solveAP(c,n::Int64,solver)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n,1:n],Bin)
    @objective(submodel,Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n) )
    
    @constraint(submodel, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(submodel, destination[j = 1:n], sum(x[1:n,j]) == 1)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

In [ ]:
function solveRAP(cminus, cplus, n::Int64, solver=CplexSolver())
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    
    
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido

    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

In [ ]:
solveRAP(cminus,cplus,10)

## Compliquemos ligeramente el ejemplo anterior

Tenemos dos puntos que podríamos mejorar:

(1) No estamos generando cortes para las soluciones fraccionales

(2) Vamos a usar la heuristic callback para generar posibles soluciones. Debido a la particularidad de la lazy cut, ésta será la encargada de insertar los cortes

La forma de implementarla va a ser un tanto "rara" y se debe a particularidades de este problema. 

Adicionalmente añadimos una "information callback"

In [ ]:
function solveRAPwithHeuristic(cminus, cplus, n::Int64, solver=CplexSolver(CPX_PARAM_SCRIND=0))
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    pending = JuMP.GenericAffExpr[]
    function heuristic(m)
        EPSILON=0.00001
        x_val = getvalue(x)
        xTest=zeros(Int64,n)
        yUsed=zeros(Int64,n)
        bestVal=0
        for i in 1:n
            val=(-1)
            bestVal=0
            for j in 1:n
                if yUsed[j]==0 && val<x_val[i,j]
                    val=x_val[i,j]
                    bestVal=j
                end
            end
            xTest[i]=bestVal
            yUsed[bestVal]=1
        end
        
        cost=deepcopy(cminus)        
        for i in 1:n
            x_val[i,:]=0
            x_val[i,xTest[i]]=1
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        evaluacionCost = - optimalCost
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
                if x_val[i,j]>EPSILON
                    evaluacionCost += cost[i,j]
                end
            end
        end
        #println("\nIn: ",MathProgBase.cbgetobj(m)," vs ",evaluacionCost)
        if(MathProgBase.cbgetobj(m)>evaluacionCost)
            push!(pending,rhs)
            setsolutionvalue(m,θ,evaluacionCost)
            for i in 1:n
                for j in 1:n
                    if x_val[i,j]>EPSILON
                        setsolutionvalue(m,x[i,j],1)
                    else
                        setsolutionvalue(m,x[i,j],0)
                    end
                end
            end
            addsolution(m)
        end
    end
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    function cutGenerator(m)
        while !isempty(pending)
            rhs = pop!(pending)
            @usercut(m,θ>=rhs)
        end
    end
    function infocallback(m)
        println("exploredNodes: ",MathProgBase.cbgetexplorednodes(m)," obj: ",MathProgBase.cbgetobj(m), " bestBound: ",MathProgBase.cbgetbestbound(m))        
    end
    #:MIPNode if at node in branch-and-cut tree, 
    #:MIPSol at an integer-feasible solution,
    #:Intermediate otherwise.
    addinfocallback(m, infocallback, when = :Intermediate)
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    addcutcallback(m,cutGenerator)
    addheuristiccallback(m,heuristic)
    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

In [ ]:
solveRAPwithHeuristic(cminus,cplus,10)